In [11]:
import logging
import time
from pprint import pprint
import ray
import random
from random import randint
import numpy as np

In [12]:
from collections import defaultdict
@ray.remote
class LoggingActor(object):
    def __init__(self):
        self.logs = defaultdict(list)
    
    def log(self, index, message):
        self.logs[index].append(message)
    
    def get_logs(self):
        return dict(self.logs)
    
@ray.remote
def run_experiment(experiment_index, logging_actor):
    for i in range(5):
        time.sleep(1)
        # Push a logging message to the actor.
        logging_actor.log.remote(experiment_index, 'On iteration {}'.format(i))  

In [13]:
if ray.is_initialized:
    ray.shutdown()
context = ray.init(logging_level=logging.ERROR)
pprint(context)

RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.12', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-23_07-58-25_331133_88314/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-23_07-58-25_331133_88314/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-05-23_07-58-25_331133_88314', 'metrics_export_port': 55200, 'gcs_address': '127.0.0.1:51209', 'address': '127.0.0.1:51209', 'node_id': '3ae3813192a1d500ac38011ab6cf3d023a59c7eb8044d95b5d83282d'})


In [14]:
logging_actor = LoggingActor.remote()
experiment_ids = []
for i in range(3):
    experiment_ids.append(run_experiment.remote(i, logging_actor))

In [15]:
logs = logging_actor.get_logs.remote()
print(ray.get(logs))

{0: ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4'], 1: ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4'], 2: ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4']}
